In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
plt.rcParams['font.family'] = 'SimSun'
data = pd.read_csv('../newData.csv',encoding='gbk')

# 转换日期特征为数值型
data['航班日期'] = pd.to_datetime(data['航班日期'])
data['航班日期'] = (data['航班日期'] - data['航班日期'].min()).dt.days

# 提取特征和标签
X = data[['航班日期', '月份', '日期', '星期几', '季节', '节假日']].values
y = data['航线金额'].values

# 归一化特征数据
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集和验证集
train_size = int(0.8 * len(data))  # 使用 80% 的数据作为训练集
X_train = X_scaled[:train_size]
y_train = y[:train_size]
X_val = X_scaled[train_size:]
y_val = y[train_size:]

# 转换数据为三维输入形状
time_steps = 7  # 使用过去 7 天的数据作为输入
num_features = X.shape[1]
X_3d = np.reshape(X, (X.shape[0], time_steps, num_features))

# 创建 LSTM 模型
def create_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units=units, dropout=dropout_rate, input_shape=(time_steps, num_features)))
    model.add(Dense(1))  # 回归任务，输出为单个值
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# 定义参数空间
param_grid = {
    'units': [32, 64, 128],  # LSTM 层的单元数
    'dropout_rate': [0.0, 0.2, 0.4],  # dropout 比率
}

# 创建 LSTM 模型
lstm_model = KerasRegressor(build_fn=create_lstm_model, epochs=10, batch_size=32, verbose=0)

# 使用网格搜索寻找最佳参数组合
grid = GridSearchCV(estimator=lstm_model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_3d, y)

# 打印最佳参数组合和得分
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

ModuleNotFoundError: No module named 'tensorflow.python.keras.wrappers'

出问题：找不到KerasRegressor，可能是tensorflow版本的问题，未解决。